## 1、为 新月形 数据集训练并微调一棵决策树。

a. 使用make_moons(n_samples=10000，noise=0.4)生成一个 新月形 数据集。

b. 使用train_test_split()拆分训练集和测试集。

c. 使用交叉验证的网格搜索（在GridSearchCV类的帮助下）为Decision-TreeClassifier找到适合的超参数值。提示：尝试max_leaf_nodes的多种值。

d. 使用超参数对整个训练集进行训练，并测量模型在测试集上的性能。你应该得到约85%～87%的准确率。


In [1]:
import numpy as np
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape

((8000, 2), (2000, 2), (8000,))

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42)

params_grid = [{
    "max_depth": [2, 4, 6, None],
    "max_leaf_nodes": [5, 7, 9]
}]

grid_search = GridSearchCV(clf, params_grid)
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid=[{'max_depth': [2, 4, 6, None],
                          'max_leaf_nodes': [5, 7, 9]}])

In [4]:
grid_search.best_params_

{'max_depth': 2, 'max_leaf_nodes': 5}

In [5]:
clf_best = DecisionTreeClassifier(max_depth=2, max_leaf_nodes=5, random_state=42)
clf_best.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2, max_leaf_nodes=5, random_state=42)

In [6]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, clf_best.predict(X_test))
print(f"最佳预测正确率为：{score * 100:.3f} %")

最佳预测正确率为：86.300 %


## 2. 按照以下步骤种植森林。

a.继续之前的练习，生产1000个训练集子集，每个子集包含随机挑选的100个实例。提示：使用Scikit-Learn的ShuffleSplit类来实现。

b.使用前面得到的最佳超参数值，在每个子集上训练一棵决策树。在测试集上评估这1000棵决策树。因为训练集更小，所以这些决策树的表现可能比第一棵决策树要差一些，只能达到约80%的精度。

c.见证奇迹的时刻到了。用每个测试集实例，生成1000棵决策树的预测，然后仅保留次数最频繁的预测［可以使用SciPy的mode()函数］。这样你在测试集上可获得大多数投票的预测结果。

d.评估测试集上的这些预测，你得到的准确率应该比第一个模型更高（高出0.5%～1.5%）。恭喜，你已经训练出了一个随机森林分类器！

In [7]:
from sklearn.model_selection import ShuffleSplit
rs = ShuffleSplit(n_splits=1000, train_size=100, random_state=42)
rs.get_n_splits(X_train, y_train)

1000

In [8]:
y_train.shape

(8000,)

In [9]:
from scipy.stats import mode
import numpy as np
accuracy_score_list = []
for train_index, test_index in rs.split(X):
    new_x_train, new_x_test = X[train_index], X[test_index]
    new_y_train, new_y_test = y[train_index], y[test_index]
    fitted_clf = DecisionTreeClassifier(max_depth=2, max_leaf_nodes=5, random_state=42).fit(new_x_train, new_y_train)
    scores = accuracy_score(new_y_test, fitted_clf.predict(new_x_test))
    accuracy_score_list.append(round(scores, 6))

In [10]:
scores_list = np.array(accuracy_score_list)
mode_info = mode(scores_list)
print(f"最佳预测正确率为：{mode_info.mode * 100:.3f} % , 出现次数为：{mode_info.count}")

最佳预测正确率为：85.263 % , 出现次数为：9
